<a href="https://colab.research.google.com/github/arpitlabs/azure-openai-client/blob/main/simple_chatbot_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the libraries

In [ ]:
import os
import gradio as gr
from openai import AzureOpenAI
from google.colab import userdata

Set up the required variables, secrets etc.

In [ ]:
MODEL = "gpt-4.1-mini"
AZURE_ENDPOINT = "https://arpitk-openai-003.openai.azure.com/"
SYSTEM_PROMPT = "You are a helpful assistant."

# AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
print(AZURE_OPENAI_KEY)
if not AZURE_OPENAI_KEY:
    raise ValueError("Please set the AZURE_OPENAI_KEY.")

Initialize AzureOpenAI client

In [ ]:
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
)

Define a function to have a conversation with the LLM using `client`

In [ ]:
def respond(message: str, chat_history):
    """Minimal handler: sends conversation to Azure OpenAI and returns updated history.

    Expects chat_history in Gradio 'messages' format (list of {'role','content'} dicts).
    """
    if not message or not message.strip():
        return chat_history or [], ""

    # Ensure history is a list; default to empty list
    chat_history = chat_history or []

    # Build messages for the API: system prompt + existing history + new user message
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + chat_history + [
        {"role": "user", "content": message}
    ]

    # Send the messages to the LLM, get the response
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.2,
        max_tokens=800,
    )

    # Simple extraction with basic fallback
    try:
        assistant_text = resp.choices[0].message.content
    except Exception:
        try:
            assistant_text = resp.choices[0].text
        except Exception:
            assistant_text = str(resp)

    # Append to history in 'messages' format and return (plus clear textbox)
    updated = chat_history + [{"role": "user", "content": message}, {"role": "assistant", "content": assistant_text}]
    return updated, ""

Initiate gradio and launch the chat interface:

In [ ]:
def reset():
    return [], ""

with gr.Blocks() as demo:
    gr.Markdown("## Minimal Azure OpenAI + Gradio Chatbot")
    chatbot = gr.Chatbot(type="messages", allow_tags=True)
    txt = gr.Textbox(show_label=False, placeholder="Type a message and press Enter")
    send = gr.Button("Send")
    clear = gr.Button("Clear")

    txt.submit(respond, [txt, chatbot], [chatbot, txt]) # Enter = submit
    send.click(respond, [txt, chatbot], [chatbot, txt]) # Send = submit
    clear.click(reset, None, [chatbot, txt])

    demo.launch()
